In [2]:
import fitz
import numpy as np
import pandas as pd
from tqdm import tqdm

import io
from PIL import Image, ImageDraw, ImageFont, ImageColor

import math

In [3]:
vol1_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf'
vol2_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf'
vol3_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf'

vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

vol1_pages = [vol1_doc[i] for i in range(vol1_doc.page_count)]
vol2_pages = [vol2_doc[i] for i in range(vol2_doc.page_count)]
vol3_pages = [vol3_doc[i] for i in range(vol3_doc.page_count)]

In [4]:
vol1_char_df = pd.read_pickle("../input/char_df/vol1_df.pkl")
vol2_char_df = pd.read_pickle("../input/char_df/vol2_df.pkl")
vol3_char_df = pd.read_pickle("../input/char_df/vol3_df.pkl")

vol1_index = list(range(616, 639)) #inclusive
vol2_index = list(range(703, 725))
vol3_index = list(range(555, 583))

In [5]:
TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

In [12]:
keep_cols = vol1_char_df.columns.difference(["char_num", "char", "char_origin", "char_bbox", "char_x0", "char_y0", "char_x1", "char_y1", "pruned_char_x0", "pruned_char_y0", "pruned_char_x1", "pruned_char_y1"], sort=False).tolist()

In [34]:
vol1_char_df

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,span_bbox,word_num,word,word_bbox,pruned_word,pruned_word_bbox,char_num,char,char_origin,char_bbox
0,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,"(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,"(110.87999725341797, 127.92001342773438, 156.9...",AUTRES,"(110.87999725341797, 127.92001342773438, 156.9...",0,A,"(110.87999725341797, 136.92001342773438)","(110.87999725341797, 127.92001342773438, 119.4..."
1,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,"(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,"(110.87999725341797, 127.92001342773438, 156.9...",AUTRES,"(110.87999725341797, 127.92001342773438, 156.9...",1,U,"(119.42528533935547, 136.92001342773438)","(119.42528533935547, 127.92001342773438, 128.0..."
2,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,"(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,"(110.87999725341797, 127.92001342773438, 156.9...",AUTRES,"(110.87999725341797, 127.92001342773438, 156.9...",2,T,"(127.97057342529297, 136.92001342773438)","(127.97057342529297, 127.92001342773438, 135.2..."
3,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,"(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,"(110.87999725341797, 127.92001342773438, 156.9...",AUTRES,"(110.87999725341797, 127.92001342773438, 156.9...",3,R,"(135.19717407226562, 136.92001342773438)","(135.19717407226562, 127.92001342773438, 143.1..."
4,1,0,0,0,"(110.87999725341797, 127.92001342773438, 344.0...",0,0,"(1.0, 0.0)","(110.87999725341797, 127.92001342773438, 344.0...",0,...,"(110.87999725341797, 127.92001342773438, 344.0...",0,AUTRES,"(110.87999725341797, 127.92001342773438, 156.9...",AUTRES,"(110.87999725341797, 127.92001342773438, 156.9...",4,E,"(143.0890655517578, 136.92001342773438)","(143.0890655517578, 127.92001342773438, 150.34..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764639,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,"(134.63999938964844, 294.22998046875, 245.3221...",3,SOIXANTE-SIX,"(197.3344268798828, 294.22998046875, 245.32212...",SOIXANTESIX,"(197.3344268798828, 294.22998046875, 245.32212...",7,E,"(227.60250854492188, 299.47998046875)","(227.60250854492188, 294.22998046875, 232.2247..."
1764640,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,"(134.63999938964844, 294.22998046875, 245.3221...",3,SOIXANTE-SIX,"(197.3344268798828, 294.22998046875, 245.32212...",SOIXANTESIX,"(197.3344268798828, 294.22998046875, 245.32212...",8,-,"(232.03839111328125, 299.47998046875)","(232.03839111328125, 294.22998046875, 234.3461..."
1764641,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,"(134.63999938964844, 294.22998046875, 245.3221...",3,SOIXANTE-SIX,"(197.3344268798828, 294.22998046875, 245.32212...",SOIXANTESIX,"(197.3344268798828, 294.22998046875, 245.32212...",9,S,"(234.15965270996094, 299.47998046875)","(234.15965270996094, 294.22998046875, 238.0137..."
1764642,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,"(134.63999938964844, 294.22998046875, 245.3221...",3,SOIXANTE-SIX,"(197.3344268798828, 294.22998046875, 245.32212...",SOIXANTESIX,"(197.3344268798828, 294.22998046875, 245.32212...",10,I,"(237.82630920410156, 299.47998046875)","(237.8263092041015

In [8]:
keep_cols = vol1_char_df.columns.difference(["char_num", "char", "char_origin", "char_bbox", "char_x0", "char_y0", "char_x1", "char_y1", "pruned_char_x0", "pruned_char_y0", "pruned_char_x1", "pruned_char_y1"], sort=False).tolist()

In [9]:
keep_cols

['vol_num',
 'page_num',
 'block_num',
 'block_num_absolute',
 'block_bbox',
 'line_num',
 'line_wmode',
 'line_dir',
 'line_bbox',
 'span_num',
 'span_size',
 'span_flags',
 'span_font',
 'span_color',
 'span_ascender',
 'span_descender',
 'span_origin',
 'span_bbox',
 'word_num',
 'word',
 'word_bbox',
 'pruned_word',
 'pruned_word_bbox']

In [11]:
all_vol_data = [(vol1_char_df, vol1_index, vol1_doc, "word_bbox_sample_vol1"),
                (vol2_char_df, vol2_index, vol2_doc, "word_bbox_sample_vol2"),
                (vol3_char_df, vol3_index, vol3_doc, "word_bbox_sample_vol3")]

for vol_char_df, vol_index, vol_doc, output_name in all_vol_data:
    image_list = []
    keep_cols = vol_char_df.columns.difference(["char_num", "char", "char_origin", "char_bbox", "char_x0", "char_y0", "char_x1", "char_y1", "pruned_char_x0", "pruned_char_y0", "pruned_char_x1", "pruned_char_y1"], sort=False).tolist()
    for page_num in tqdm(vol_index):
        pix_map = vol_doc.get_page_pixmap(page_num,matrix=mat)
        image = Image.open(io.BytesIO(pix_map.tobytes()))
        draw = ImageDraw.Draw(image)

        temp_df = vol_char_df[vol_char_df["page_num"] == page_num].loc[:, keep_cols].drop_duplicates()

        for coord in temp_df['pruned_word_bbox'] :
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#FF7F50"), width=5)

        for coord in temp_df['word_bbox']:
            x0, y0, x1, y1 = [f*TARGET_DPI/ 72 for f in coord]
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#003399"), width=2)
            
        image_list.append(image)
        #save pages of the volume
    image_list[0].save('../output/local/'+output_name+'.pdf' ,save_all=True, append_images=image_list[1:])


100%|██████████| 28/28 [00:04<00:00,  5.80it/s]
